In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import datetime
from libs.sequences import SeismicSequence
from libs.iris import irisRequests
from libs.distributions import Weibull, WeibullMM, GaussianMM, InterTimeDistribution
from importlib import reload  # Python 3.4+
import matplotlib.pyplot as plt

# Model

In [14]:
def get_nll_loss(inter_times : torch.Tensor, seq_lengths : torch.Tensor, inter_time_distr : InterTimeDistribution):
    log_prob = inter_time_distr.get_log_prob(inter_times)
    mask = SeismicSequence.pack_sequences_mask(seq_lengths, inter_times.shape[1])
    log_like = (log_prob * mask).sum(-1)  # (N,)
    log_surv = inter_time_distr.log_survival(inter_times)  # (N, L)
    end_idx = torch.unsqueeze(seq_lengths,-1)  # (N, 1)
    log_surv_last = torch.gather(log_surv, dim=-1, index=end_idx)  # (N, 1)
    log_like += log_surv_last.squeeze(-1)  # (N,)
    return -log_like

In [15]:
class GRUPointProcess(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_mixtures=1):
        super().__init__()
        self.rnn = nn.GRU(input_dim,hidden_dim,
                          num_layers=1,
                          batch_first=True)
        self.n_mixtures = n_mixtures
        if(self.n_mixtures==1):
            self.weibull_mod = nn.Sequential(
                nn.Linear(hidden_dim, 2),
                nn.Softplus()
            )
        else:
            self.weibull_mod = nn.Sequential(
                nn.Linear(hidden_dim, self.n_mixtures, 2),
                nn.Softplus()
            )
            self.mixture_mod = nn.Sequential(
                nn.Linear(hidden_dim, self.n_mixtures),
                nn.Softmax(dim=1)
            )
    def forward(self,x):
        rnn_output, _ = self.rnn(x)
        # shift forward along the time dimension and pad
        context = F.pad(rnn_output[:, :-1, :], (0,0, 1,0))
        weibull_params = self.weibull_mod(context)
        if(self.n_mixtures == 1):
            distr = Weibull(weibull_params[..., 0], weibull_params[..., 1])
        else:
            pip_params = self.mixture_mod(context)
            distr = WeibullMM(pip_params, weibull_params[..., 0], weibull_params[..., 1])
        return context, distr

In [16]:
model = GRUPointProcess(2, 10, n_mixtures=30)

In [17]:
input_time_features = torch.cat([inter_times.unsqueeze(-1), inter_times.log().unsqueeze(-1)], dim=-1)

In [19]:
context, distr = model(input_time_features)

# Data

In [4]:
# specify regions limits
regions = {}
regions['greece'] = (30, 45,18, 44)
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)

In [5]:
load_cat = True
region_name = "japan"
region = regions[region_name]
if(load_cat):
    df = pd.read_csv("catalogs/" + region_name + ".csv", parse_dates=['Time'])
else:
    start_time = datetime.datetime(1980, 1, 1, 0, 0, 0)
    end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
    download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#")
    df.Time = pd.to_datetime(df.Time, errors='coerce')
    df.dropna(axis=0, inplace=True)
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.to_csv("catalogs/japan.csv", index=False)

/tmp/ipykernel_18028/3562696496.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("catalogs/" + region_name + ".csv", parse_dates=['Time'])


In [6]:
full_catalog = SeismicSequence.from_pandas_df(df, unit='s')

In [7]:
seqs = []
use_random_length = True
duration_days = 3
duration = 60*60*24*duration_days # in seconds
t_start = 0
max_t_end = full_catalog.t_end
while(True):
    if(use_random_length):
        t_end = t_start + np.random.exponential()*duration
    else:
        t_end = t_start + duration
    if(t_end > max_t_end):
        break
    sub_seq = full_catalog.get_subsequence(t_start, t_end)
    if(len(sub_seq.arrival_times) > 0):
        seqs.append(sub_seq)
        print(len(sub_seq.arrival_times))
    t_start = t_end
        

2
5
2
1
1
8
6
13
8
2
4
13
12
9
2
10
26
1
6
6
16
57
3
6
3
17
38
6
6
35
28
14
5
2
4
8
1
9
24
3
2
1
6
4
26
10
1
7
5
2
2
14
12
2
7
10
5
4
26
17
3
3
24
10
3
14
2
2
1
10
7
2
6
2
5
1
4
12
1
8
3
2
6
17
15
21
14
3
4
1
14
1
8
16
1
2
2
42
12
12
3
2
12
9
6
24
5
38
14
14
2
2
3
19
7
17
2
14
6
8
4
5
22
4
19
3
4
3
6
49
14
1
10
22
5
6
12
8
6
7
1
10
5
2
2
3
16
11
2
14
1
1
2
27
10
4
1
2
1
2
28
4
3
7
5
7
10
2
21
22
2
3
8
4
8
12
3
8
7
10
1
2
1
10
7
2
3
1
1
3
1
1
3
1
1
3
1
26
18
11
1
1
1
8
4
2
2
43
16
2
4
1
5
38
13
1
8
5
10
8
11
6
13
3
24
4
13
2
12
9
6
23
17
48
17
16
1
30
6
7
15
2
5
2
17
9
12
7
6
1
21
20
10
13
25
2
1
2
2
15
10
2
1
5
3
9
50
9
5
5
2
1
6
6
23
4
4
26
10
2
17
15
12
1
9
4
9
4
9
2
1
8
9
18
4
26
4
2
10
3
3
4
7
3
3
8
7
4
3
11
1
2
9
8
4
1
7
12
9
3
36
21
16
8
6
9
5
1
7
49
10
4
4
2
8
1
6
18
1
5
4
1
3
5
9
14
5
7
11
10
8
1
7
13
9
10
8
7
20
2
11
13
80
19
12
28
44
5
22
11
3
18
6
70
2
27
19
4
11
9
33
3
5
43
7
11
15
3
16
9
1
15
3
2
9
2
9
4
4
4
1
13
7
12
5
1
18
4
10
2
4
6
8
1
6
6
43
3
26
1
8
7
1
2
1
11
5
10
1

In [8]:
inter_times, features, lengths = SeismicSequence.pack_sequences(seqs)